<a href="https://colab.research.google.com/github/jayjayppark/aix_deep_learning/blob/new-branch/final_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

with open("adult_data.json", "r") as readit:
    x = json.load(readit)

df1 = pd.DataFrame(x)

with open("adult_test.json", "r") as readit:
    x = json.load(readit)

df2 = pd.DataFrame(x)

# 데이터 작성자가 임의로 train, test데이터로 나눠놨던것을 다시 합치자.
df = pd.concat([df1, df2])
df.info()
# info()로 확인해보니 nan인 데이터가 한개도 없다. 수치형데이터 6개 범주형데이터 9개가 있는것을 확인

In [ ]:
# 데이터를 확인해보니 곳곳에 ?값이 확인되었다.
df

In [ ]:
# 수치형 데이터들의 요약통계량 확인
df.describe()
# 나이는 17세부터 90세까지 있는것을 확인할수 있다. 일주일에 일하는 시간은 1시간부터 99시간 까지있고 평균은 40시간 이다.

In [ ]:
# nan값의 갯수를 확인해보니 역시 없다고 나오지만 아까전에 ?값을 확인했으므로 ?값이 nan값이나 마찬가지다.
df.isnull().sum()

In [ ]:
# ?를 nan값으로 바꿔주자.
import numpy as np

df[df == '?'] = np.nan

In [ ]:
df.isnull().sum()

In [ ]:
# nan값이 있는 행은 전부 지워준다. 총 3,620개를 지워 45,222개의 데이터가 남았다.
df = df.dropna(axis = 0)
df.info()

In [ ]:
# class값이 이상하게 '<=50K', '>50K', '<=50K.', '>50K.' 뒤에 점이 붙은상태로 한개씩 또 있다. 바꿔주자
df['class'].unique()

In [ ]:
df = df.replace({'class' : '<=50K.'}, '<=50K')
df = df.replace({'class' : '>50K.'}, '>50K')

In [ ]:
# 5만달러 이상버는 사람은 1로, 이하로 버는사람은 0으로 바꿔준다.
df = df.replace({'class' : '<=50K'}, 0)
df = df.replace({'class' : '>50K'}, 1)

In [ ]:
df['class'].unique()

In [ ]:
# 5만달러 이상버는 사람의 비율을 확인해보니 24.78퍼센트를 차지하고 있었다.
a = df['class'].value_counts()
print(a)
income_rate = round(df['class'].value_counts()[1]/len(df) * 100, 2)
print('income_rate', income_rate, '% of the dataset')

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (8,5))
plt.title('Income Distribution of Adults', fontsize=18, fontweight='bold')
percentage = df['class'].value_counts(normalize = True).rename_axis('income').reset_index(name = 'Percentage')

ax = sns.barplot(x = 'income', y = 'Percentage', data = percentage.head(10), palette='Greens_r')
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.annotate(f'{height:.0%}', (x + width/2, y + height*1.02), ha='center', fontweight='bold')

In [ ]:
''' 이 연구의 목적은 사람들의 연수입 데이터를 보면서 어떤 요인이 고소득과 관련이 있는지를 확인해보고, 돈을많이 벌기위한 힌트를 얻는것이므로,
바꿀수 없는 요인 ex)인종, 나라 ,성별 등은 제거해주겠다. 또한 education과 education-num이 겹치므로 education은 삭제해준다.
'''

In [ ]:
df = df.drop(['education', 'race', 'sex', 'native-country', 'fnlwgt'], axis=1)

In [ ]:
# 결혼상태 컬럼을 보면 너무 상태가 많고, 나중에 라벨인코더로 숫자로 바꿔줄거니 간단하게 결혼함(1)과 싱글(0) 두개의 범주로 나눠준다
df["marital-status"].unique()

In [ ]:
df["marital-status"] = df["marital-status"].replace(['Never-married','Divorced','Separated','Widowed'], 0)
df["marital-status"] = df["marital-status"].replace(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse'], 1)

In [ ]:
df["marital-status"].unique()

In [ ]:
correlation_matrix = df.corr().round(2)
sns.heatmap(data=correlation_matrix, annot=True)

plt.show()
# heatmap을 통해 수치형 데이터들과 class의 상관관계를 확인해보았다.
# 대부분 양의 상관관계를 가짐을 알수있다.
# 수입과 상관관계가 가장 큰 순서는 결혼여부(혼자사는지 여부)와 교육 수준, 나이, 주에 일하는 시간, 자본 소득, 자본 손해 순서였다.

In [ ]:
''' 다음으로 두번째로 상관계수가 큰 교육수준과 연수입의 관계를 그래프로 그려서 분석해보자.
1번부터 16번까지 순서는 아래와 같다.
 ['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th', 'HS-grad ',
 'Some-college', 'Bachelors', 'Masters', 'Doctorate', 'Prof-school', 'Assoc-acdm', 'Assoc-voc']
 '''
# 교육 수준이 높아질수록 5만달러 이상버는 사람의 비율이 높아지는것을 확인할수 있다.
# 특히 석사에서 박사로 넘어갈때 5만달러 이상버는 사람의 퍼센트가 크게 뛰고 그 다음부터도 큰 폭으로 비율이 증가하는것을 확인할수있다.
percent = sns.catplot(x="education-num",y="class",data = df, kind = "bar", palette = "muted")
percent.despine(left=True)
percent = percent.set_ylabels(">50K probability")

In [ ]:
df.info()

In [ ]:
# 수업시간에 배운 KNN모델과 random forest모델들을 써보고 두 모델의 예측성능을 비교해보겠다.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

X = df.drop("class",axis=1)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)
#train test set의 class비율을 stratify = y를 함으로서 일정하게 맞춰준다.

# knn으로 분류하기 위해 범주형 변수들을 LabelEncoder로 인코딩한다.
categorical = ['workclass', 'occupation', 'relationship']

for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

# StandardScaler로 데이터를 표준화한다.
scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

# 2가지 머신러닝 모델을 불러온다.
knn_clf = KNeighborsClassifier(n_neighbors=5)
rf_clf = RandomForestClassifier(random_state = 13, n_jobs=-1, n_estimators=100)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score

# 실제값과 예측값을 넣으면 accuracy, precision, recall, f1 score, roc_auc score를 구해주는 함수
def get_clf_eval(y_test, pred):
    acc = accuracy_score(y_test , pred)
    pre = precision_score(y_test , pred)
    re = recall_score(y_test , pred)
    f1 = f1_score(y_test, pred)
    auc = roc_auc_score(y_test, pred)

    return acc, pre, re, f1, auc

In [ ]:
#모델과 train_test split한 데이터들을 넣어주면 성능평가 지표들을 구해주는 함수
def get_result(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    return get_clf_eval(y_test, pred)

In [ ]:
# 두 모델의 평가지표를 확인해보니 random forest 모델이 정확도도 더 높고 f1 score도 높아 정밀도와 재현율또한 더 높게 측정되었다.
models = [knn_clf, rf_clf]
model_names = ['KNeighbors', 'RandomForest']

# 모델과 모델이름, train_test_split 데이터를 넣으면 성능평가지표를 모델별로 표로 보여주는 함수
def get_result_pd(models, model_names, X_train, y_train, X_test, y_test):
    col_names = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    tmp = []

    for model in models:
        tmp.append(get_result(model, X_train, y_train, X_test, y_test))
    return pd.DataFrame(tmp, columns=col_names, index=model_names)

results = get_result_pd(models, model_names, X_train, y_train, X_test, y_test)
results

In [ ]:
from sklearn.metrics import roc_curve

# 모델과 모델이름, test데이터를 넣으면 roc_curve를 그려주는 함수
def draw_roc_curve(models, model_names, X_test, y_test):
    plt.figure(figsize=(10,10))

    for model in range(len(models)):
        pred = models[model].predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, pred)
        plt.plot(fpr, tpr, label=model_names[model])

    plt.plot([0,1], [0,1], 'k--', label='random quess')
    plt.title('ROC')
    plt.legend()
    plt.grid()
    plt.show()

draw_roc_curve(models, model_names, X_test, y_test)

In [ ]:
# random forest에 대해서 precision-recall curve 출력해보기
from sklearn.metrics import precision_recall_curve

pred_proba_c1 = rf_clf.predict_proba(X_test)[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)

plt.figure(figsize = (8, 6))
plt.plot(thresholds, precisions[:len(thresholds)],
         linestyle = '--', label = 'precision')
plt.plot(thresholds, recalls[:len(thresholds)], label = 'recall')
plt.xlabel('Threshold value')
plt.legend(); plt.grid()
plt.show()

In [ ]:
# gridsearchcv를 통해 random forest모델의 성능개선해보기

from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [ 6, 8 ,10],
    'n_estimators' : [50, 100, 200],
    'min_samples_split' : [8, 12]
}

rf_clf = RandomForestClassifier(random_state=13, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train.values.reshape(-1,))

In [ ]:
cv_results_df = pd.DataFrame(grid_cv.cv_results_)
cv_results_df.columns

In [ ]:
target_col = ['rank_test_score', 'mean_test_score', 'param_n_estimators', 'param_max_depth', 'param_min_samples_split']
cv_results_df[target_col].sort_values('rank_test_score').head()

In [ ]:
grid_cv.best_params_

In [ ]:
grid_cv.best_score_

In [ ]:
# grid search로 하이퍼파라미터 값을 변경해보며 최적의 파라미터를 적용해봤지만 accuracy는 별차이가 없었다.
rf_clf_best = grid_cv.best_estimator_
rf_clf_best.fit(X_train, y_train.values.reshape(-1,))

pred1 = rf_clf_best.predict(X_test)

accuracy_score(y_test , pred1)

In [ ]:
# 나머지 지표도 확인해보니 precision만 올라가고 다 조금씩 안좋아진것을 확인 할 수있었다.
# 하지만 내가 생각하기에 5만달러가 안넘는 사람을 넘는다고 분류하는것이 더 안좋으므로 presion이 더 중요하다고 할수 있다.
# 따라서 gridsearch로 찾은 best 모델을 이용하겠다.
models = [knn_clf, rf_clf_best]
model_names = ['KNeighbors', 'RandomForest']

def get_result_pd(models, model_names, X_train, y_train, X_test, y_test):
    col_names = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    tmp = []

    for model in models:
        tmp.append(get_result(model, X_train, y_train, X_test, y_test))
    return pd.DataFrame(tmp, columns=col_names, index=model_names)

results = get_result_pd(models, model_names, X_train, y_train, X_test, y_test)
results

In [ ]:
df['hours-per-week'].unique()

In [ ]:
# 마지막으로 미래의 나의 데이터를 최종 모델에 집어넣어 연수입이 5만달러 이상일지 예측해보자!!

me = pd.DataFrame([[35, 'Self-emp-inc', 11, 1, 'Tech-support', 'Husband', 200000, 0, 60]], columns = X_train.columns)


for feature in categorical:
        le = preprocessing.LabelEncoder()
        le.fit(df[feature])
        me[feature] = le.transform(me[feature])

me = pd.DataFrame(scaler.transform(me), columns = X.columns)
pred_me = rf_clf_best.predict(me)
print(pred_me)
# 결과값은 1으로 연수입이 5만달러 이상이라고 예측하였습니다! 정확한 예측모델이군!!

In [ ]:
X_train

In [ ]:
me

In [ ]:
X

In [ ]:
X_train.columns[:]

In [ ]:
categorical

In [ ]:
df